![Flujo de Data Profiling](asset/raw_dataprofiling_01.png)

El **Data Profiling** tiene como objetivo comprender en profundidad un dataset en su estado inicial (**RAW / PRE**), antes de aplicar transformaciones, modelado o analítica avanzada.

En este proyecto, el proceso se aplica sobre el archivo **`dirty_cafe_sales.csv`**, como parte del proyecto:

**📦 CoffeeSales – Data Profiling (RAW PRE)**

A través de este análisis buscamos:

- Comprender la **estructura real** del dataset recibido.
- Identificar **problemas de calidad de datos** (nulos, duplicados, valores inválidos).
- Evaluar la **consistencia de tipos y formatos**.
- Detectar posibles **reglas de negocio incumplidas**.
- Determinar si el dataset está en condiciones de avanzar a etapas posteriores del pipeline (**decisión GO / NO-GO**).

Este proceso se ejecuta sobre la capa **RAW PRE**, sin aplicar transformaciones, con el fin de obtener un diagnóstico fiel del estado original de los datos.

---

## 🧪 Consultas utilizadas en la exploración
**Proyecto:** CoffeeSales – Data Profiling (RAW PRE)
**Dataset analizado:** `dirty_cafe_sales.csv`

Las siguientes consultas SQL se utilizan para realizar una exploración estructurada y repetible del dataset:

- **Vista inicial del dataset**
  _Permite revisar las primeras filas para comprender la estructura, tipos de datos y valores generales._

- **Volumen total de registros**
  _Valida la cantidad total de filas presentes en `dirty_cafe_sales.csv`._

- **Conteo de valores nulos por columna**
  _Evalúa la presencia y distribución de valores nulos en cada campo._

- **Detección de registros duplicados**
  _Identifica filas o claves repetidas que pueden afectar la integridad del análisis._

- **Unicidad y cardinalidad de columnas clave**
  _Analiza si ciertos campos cumplen su rol como identificadores o dimensiones._

- **Validación de formatos de fecha**
  _Revisa consistencia en campos de fecha y timestamps._

- **Revisión de valores inválidos o fuera de dominio**
  _Detecta valores como `NaN`, `UNKNOWN`, `ERROR` o inconsistencias similares._

- **Análisis de longitud de campos de texto**
  _Evalúa longitudes para identificar truncamientos, valores vacíos o problemas de diseño._

- **Detección de espacios y caracteres no deseados**
  _Identifica espacios en blanco, saltos de línea u otros caracteres sucios._

- **Resumen general del dataset**
  _Consolida métricas clave como totales, rangos de fechas y valores distintos._

---

## 🧠 Resultado esperado

El resultado de este proceso es un **diagnóstico técnico documentado** del dataset **`dirty_cafe_sales.csv`**, que permite:

- Justificar una decisión **GO / NO-GO**.
- Definir las transformaciones necesarias para etapas posteriores.
- Sentar la base técnica para **modelado, ETL y arquitectura Medallion**.

![Flujo de Data Profiling](asset/01_vista_inicial_dataset.png)

In [1]:
%%sql
-- ========================================
-- EDA_SAMPLE_ROWS
-- Descripción: Ver una muestra de filas para entender el contenido
-- ========================================

SELECT *
FROM profiling.data_profiling_summary
LIMIT 10;

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


![Flujo de Data Profiling](asset/02_estructura_tabla.png)

In [2]:
%%sql
-- ========================================
-- TEMPLATE: RAW_DESCRIBE_TABLE
-- Descripción: Mostrar estructura (columnas y tipos) de una tabla
-- ========================================

DESCRIBE profiling.data_profiling_summary;

,column_name,column_type,null,key,default,extra
0,Transaction ID,VARCHAR,YES,NaN,NaN,NaN
1,Item,VARCHAR,YES,NaN,NaN,NaN
2,Quantity,VARCHAR,YES,NaN,NaN,NaN
3,Price Per Unit,VARCHAR,YES,NaN,NaN,NaN
4,Total Spent,VARCHAR,YES,NaN,NaN,NaN
5,Payment Method,VARCHAR,YES,NaN,NaN,NaN
6,Location,VARCHAR,YES,NaN,NaN,NaN
7,Transaction Date,VARCHAR,YES,NaN,NaN,NaN


![Flujo de Data Profiling](asset/03_contar_nulos_pseudo.png)

In [3]:
%%sql
-- ========================================
-- TEMPLATE: EDA_NULL_PSEUDONULL_COUNT
-- Descripción: Contar NULL reales y pseudo-NULL ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR') por columna
-- ========================================

SELECT
    -- Columna 1
    SUM(CASE
            WHEN "Transaction ID" IS NULL
              OR TRIM("Transaction ID") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Transaction_ID_null_like,

    -- Columna 2
    SUM(CASE
            WHEN Item IS NULL
              OR TRIM(Item) IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Item_null_like,

    -- Columna 3
    SUM(CASE
            WHEN Quantity IS NULL
              OR TRIM(Quantity) IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Quantity_null_like,

    -- Columna 4
    SUM(CASE
            WHEN "Price Per Unit" IS NULL
              OR TRIM("Price Per Unit") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Price_Per_Unit_null_like,

    -- Columna 5
    SUM(CASE
            WHEN "Total Spent" IS NULL
              OR TRIM("Total Spent") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Total_Spent_null_like,

    -- Columna 6
    SUM(CASE
            WHEN "Payment Method" IS NULL
              OR TRIM("Payment Method") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Payment_Method_null_like,

    -- Columna 7
    SUM(CASE
            WHEN Location IS NULL
              OR TRIM(Location) IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Location_null_like,

    -- Columna 8
    SUM(CASE
            WHEN "Transaction Date" IS NULL
              OR TRIM("Transaction Date") IN ('', 'NA', 'NaN', 'UNKNOWN', 'ERROR')
        THEN 1 ELSE 0 END) AS Transaction_Date_null_like

FROM profiling.data_profiling_summary;

,Transaction_ID_null_like,Item_null_like,Quantity_null_like,Price_Per_Unit_null_like,Total_Spent_null_like,Payment_Method_null_like,Location_null_like,Transaction_Date_null_like
0,0,969,479,533,502,3178,3961,460
